In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as f 
from torch.autograd import Variable
torch.backends.cudnn.deterministic = True
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random 
from matplotlib import pyplot as plt
from sklearn import decomposition
from sklearn import manifold
from scipy import stats
from itertools import cycle
import sys
import utilities as ut
from Network import Net
import Network as nn
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
import matplotlib

matplotlib.rcParams["figure.dpi"] = 100


use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

#ab y c con 3000 samples, 256 batch y 10000 objetos. 

results = []
num_classes = 2
learning_rate = 0.001
samples = 3000 #
epsilon=0
#for epsilon in [0.1, 0.05, 0.025, 0.15]:
for sc in ['_RRab', '_RRc']: #,'_RRc','_RRab'
    for batch_size in [256]:
        for hidden_size in [100]:
            for aux_loss_activated in [True]:
                for EPS1 in [0.005]:
                    for n in [5000, 10000]:
                        for opt in [1]:
                            for t in range(15):
                                train_dataset, test_dataset = ut.load_files(dataset=1, subclass=sc)
                                input_size = train_dataset.shape[1]-1

                                if n < 50000:
                                    train_dataset = ut.down_sampling(train_dataset)
                                    train_dataset = train_dataset.sample(n)
                                    print(train_dataset)
                                else: 
                                    trainig_dataset_a = train_dataset[train_dataset.label=='ClassA']
                                    print('shape: ', trainig_dataset_a.shape[0])
                                    n2 = n - trainig_dataset_a.shape[0]
                                    print('clase no RR Lrae', n2)
                                    trainig_dataset_b = train_dataset[~(train_dataset.label=='ClassA')].sample(n2)
                                    train_dataset = pd.concat([trainig_dataset_a, trainig_dataset_b])

                                train_dataset, _ = ut.delete_outliers(train_dataset, test_dataset)

                                train_dataset = ut.sort_columns(train_dataset)
                                test_dataset = ut.sort_columns(test_dataset)

                                test_dataset_pred = test_dataset.copy()
                                train_dataset_pred = train_dataset.copy()

                                try:
                                    data_prior = ut.generate_samples_2D(samples, train_dataset, distribution='uniform')

                                    train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=42)

                                    train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.2, random_state=42)
                                    print(train_dataset_prior.columns)
                                    _, _, train_target_prior, train_loader_prior = ut.get_tensors(train_dataset_prior, batch_size)
                                    _, _, val_target_prior, val_loader_prior     = ut.get_tensors(val_dataset_prior, batch_size)
                                    _, _, train_target, train_loader             = ut.get_tensors(train_dataset, batch_size)
                                    _, _, train_target_pred, train_loader_pred   = ut.get_tensors(train_dataset_pred, batch_size)
                                    _, _, val_target, val_loader                 = ut.get_tensors(val_dataset_prior, batch_size)
                                    _, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)
                                    _, _, test_target_pred, test_loader_pred     = ut.get_tensors(test_dataset_pred, batch_size)

                                    net = Net(input_size, hidden_size, hidden_size, num_classes)
                                    net.cuda()

                                    hist_val, hist_train = nn.train(net, train_loader, train_loader_prior, val_loader, test_loader,
                                    EPS1, learning_rate, input_size, aux_loss_activated=aux_loss_activated, patience=10)

                                    acc_train, recall_train, f1_train = nn.get_results(net, train_loader, input_size)
                                    acc_test, recall_test, f1_test  = nn.get_results(net, test_loader, input_size)
                                    results.append([acc_train, acc_test,recall_train, recall_test, f1_train, f1_test, epsilon, batch_size, hidden_size, aux_loss_activated, EPS1, n, opt, sc])
                                    pd.DataFrame(results, columns=['acc_train', 'acc_test','recall_train', 'recall_test','f1_train', 'f1_test',
                                     'epsilon', 'batch_size', 'hidden_size',
                                     'aux_loss_activated', 'EPS1', 'n', 'opt', 'sc']).to_csv('subclasses_07-05-2022-1D-period.csv')
                                except Exception as e:
                                    print(e) 
                                    print(str(batch_size)+"-"+str(hidden_size)+"-"+str(aux_loss_activated)+"-"+str(EPS1))

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/franciscoperez/anacon

Freq1_harmonics_rel_phase_0
Freq2_harmonics_rel_phase_0
Freq3_harmonics_rel_phase_0
downsampling
21885
369616
done downsampling
        Amplitude  AndersonDarling  Autocor_length  Beyond1Std    CAR_mean  \
5472       0.2755         0.229660               1    0.353333   23.520652   
59067      0.3450         0.000116               1    0.370000   55.028761   
325888     0.2405         0.003151               1    0.353333   26.861221   
315033     0.0540         0.000000               1    0.173333  193.960646   
67839      0.1390         0.000000              11    0.206667    0.071036   
...           ...              ...             ...         ...         ...   
88103      0.0495         0.060294               4    0.338983   58.234361   
85233      0.1010         0.000000               7    0.220000   32.254755   
342154     0.0185         0.147566               2    0.333333   15.465235   
95854      0.5475         0.000000              13    0.107023    0.010237   
221400     0.3

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Epochs:  1000
Epoch:  0
training: epoch:  1  loss:  8.056302785873413 -- aux loss:  159.97914463281631
validating: epoch:  1  loss:  3.0166909098625183
Epoch:  1
training: epoch:  2  loss:  7.2409802079200745 -- aux loss:  136.7267078757286
validating: epoch:  2  loss:  2.5482908487319946
Epoch:  2
training: epoch:  3  loss:  7.038043916225433 -- aux loss:  115.75536966323853
validating: epoch:  3  loss:  2.183373361825943
Epoch:  3
training: epoch:  4  loss:  6.45682156085968 -- aux loss:  101.491778165102
validating: epoch:  4  loss:  1.9522803127765656
Epoch:  4
training: epoch:  5  loss:  6.56568568944931 -- aux loss:  92.54642680287361
validating: epoch:  5  loss:  1.8161939978599548
Epoch:  5
training: epoch:  6  loss:  6.034607917070389 -- aux loss:  87.4308622777462
validating: epoch:  6  loss:  1.734245777130127
Epoch:  6
training: epoch:  7  loss:  5.748412877321243 -- aux loss:  84.20770829916
validating: epoch:  7  loss:  1.6822834312915802
Epoch:  7
training: epoch:  8  lo

training: epoch:  41  loss:  4.828803658485413 -- aux loss:  77.39144718647003
validating: epoch:  41  loss:  1.5666247010231018
Epoch:  41
training: epoch:  42  loss:  4.548972964286804 -- aux loss:  77.39032486081123
validating: epoch:  42  loss:  1.5665830969810486
Epoch:  42
training: epoch:  43  loss:  4.3839854300022125 -- aux loss:  77.38799798488617
validating: epoch:  43  loss:  1.5665431916713715
Epoch:  43
training: epoch:  44  loss:  4.391044974327087 -- aux loss:  77.38632303476334
validating: epoch:  44  loss:  1.5665090382099152
Epoch:  44
training: epoch:  45  loss:  4.404694110155106 -- aux loss:  77.38489046692848
validating: epoch:  45  loss:  1.5664867460727692
Epoch:  45
training: epoch:  46  loss:  4.409706473350525 -- aux loss:  77.3838158249855
validating: epoch:  46  loss:  1.5664642751216888
Epoch:  46
training: epoch:  47  loss:  4.5069955587387085 -- aux loss:  77.38280910253525
validating: epoch:  47  loss:  1.566444456577301
Epoch:  47
training: epoch:  48

training: epoch:  81  loss:  4.535919219255447 -- aux loss:  77.37570121884346
validating: epoch:  81  loss:  1.5663098394870758
Epoch:  81
training: epoch:  82  loss:  4.462322771549225 -- aux loss:  77.37570118904114
validating: epoch:  82  loss:  1.5663098394870758
Epoch:  82
training: epoch:  83  loss:  4.406862139701843 -- aux loss:  77.37570115923882
validating: epoch:  83  loss:  1.566309779882431
Epoch:  83
training: epoch:  84  loss:  4.491509795188904 -- aux loss:  77.37568879127502
validating: epoch:  84  loss:  1.5663093626499176
Epoch:  84
training: epoch:  85  loss:  4.63693031668663 -- aux loss:  77.37568607926369
validating: epoch:  85  loss:  1.5663093626499176
Epoch:  85
training: epoch:  86  loss:  4.417659282684326 -- aux loss:  77.37568607926369
validating: epoch:  86  loss:  1.5663093328475952
Epoch:  86
training: epoch:  87  loss:  4.495303213596344 -- aux loss:  77.37568607926369
validating: epoch:  87  loss:  1.566309243440628
Epoch:  87
training: epoch:  88  l

recall
tensor(0.9687)
precision
tensor(0.3547)
f1_score
tensor(0.5192)
Accuracy of the network on test objects: 35 %
35.46509
sum mask2 - L1:  tensor(5784, device='cuda:0')
sum mask2 - L2:  tensor(9475, device='cuda:0')
sum mask2 - L3:  tensor(191, device='cuda:0')
sum mask1 - L1 (aux):  tensor(216, device='cuda:0')
sum mask1 - L2 (aux):  tensor(525, device='cuda:0')
sum mask1 - L3 (aux):  tensor(9, device='cuda:0')
Epoch:  120
training: epoch:  121  loss:  4.496736764907837 -- aux loss:  77.37562254071236
validating: epoch:  121  loss:  1.5663080513477325
Epoch:  121
training: epoch:  122  loss:  4.448580771684647 -- aux loss:  77.37562254071236
validating: epoch:  122  loss:  1.5663080513477325
Epoch:  122
training: epoch:  123  loss:  4.508920431137085 -- aux loss:  77.37562254071236
validating: epoch:  123  loss:  1.5663080513477325
Epoch:  123
training: epoch:  124  loss:  4.483153849840164 -- aux loss:  77.37562254071236
validating: epoch:  124  loss:  1.5663080513477325
Epoch:  

In [ ]:
nn.get_results(net, train_loader, input_size)
nn.get_results(net, test_loader, input_size)

In [ ]:
nn.get_roc_curve(net, test_loader, input_size, title="Regularization")

In [ ]:
torch.save(net.state_dict(), 'model.pkl')

In [ ]:
ut.plot_training(hist_val, hist_train)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))
curves, labels, probs_test = ut.get_predictions(net, test_loader_pred, device)
pred_labels = probs_test.argmax(1, keepdim = True)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax)

In [ ]:
#csv_file = open("size_MLP_noise.csv", "a")
#csv_file.write(str(np.asarray(acc_testing))+","+str(np.asarray(acc_training))+","+str(samples)+","+str(epsilon)+","+str(n)+","+str(hidden_size)+"\n")
#csv_file.close()

In [ ]:
outputs, intermediates, intermediates2, labels = ut.get_representations(net, train_loader, device)
outputs_test, intermediates_test, intermediates2_test, labels_test = ut.get_representations(net, test_loader, device)

In [ ]:
intermediate_pca_data, intermediate_pca_data_test = ut.get_pca(intermediates, data_test=intermediates_test)
fig, axs = plt.subplots(2, 3, figsize=(20,15))
ut.plot_representations(intermediate_pca_data, labels, axs[0, 0])
ut.plot_representations(intermediate_pca_data_test, labels_test, axs[1, 0])
intermediate2_pca_data, intermediate2_pca_data_test = ut.get_pca(intermediates2, data_test=intermediates2_test)
ut.plot_representations(intermediate2_pca_data, labels, axs[0, 1])
ut.plot_representations(intermediate2_pca_data_test, labels_test, axs[1, 1])
output_pca_data, output_pca_data_test = ut.get_pca(outputs, data_test=outputs_test)
ut.plot_representations(output_pca_data, labels, axs[0, 2])
ut.plot_representations(output_pca_data_test, labels_test, axs[1, 2])

In [ ]:
import umap

fig, axs = plt.subplots(2, 3, figsize=(20,15))
print('first')
reducer = umap.UMAP()
print('first ..')
mapper_train = reducer.fit_transform(intermediates.numpy())
print('first ...')
reducer = umap.UMAP()
mapper_test = reducer.fit_transform(intermediates_test.numpy())
ut.plot_representations(mapper_train, labels, axs[0, 0])
ut.plot_representations(mapper_test, labels_test, axs[1, 0])

reducer = umap.UMAP()
mapper_train = reducer.fit_transform(intermediates2.numpy())
reducer = umap.UMAP()
mapper_test = reducer.fit_transform(intermediates2_test.numpy())
ut.plot_representations(mapper_train, labels, axs[0, 1])
ut.plot_representations(mapper_test, labels_test, axs[1, 1])

reducer = umap.UMAP()
mapper_train = reducer.fit_transform(outputs.numpy())
reducer = umap.UMAP()
mapper_test = reducer.fit_transform(outputs_test.numpy())

ut.plot_representations(mapper_train, labels, axs[0, 2])
ut.plot_representations(mapper_test, labels_test, axs[1, 2])

In [ ]:
N_CURVES = 25000

fig, axs = plt.subplots(2, 3, figsize=(20,15))
intermediate_tsne_data, intermediate_tsne_data_test = ut.get_tsne(intermediates, data_test= intermediates_test, n_curves = N_CURVES)
ut.plot_representations(intermediate_tsne_data, labels, axs[0, 0],  n_curves = N_CURVES)
ut.plot_representations(intermediate_tsne_data_test, labels_test, axs[1, 0], n_curves = N_CURVES)

intermediate2_tsne_data, intermediate2_tsne_data_test = ut.get_tsne(intermediates2, data_test=intermediates2_test, n_curves = N_CURVES)
ut.plot_representations(intermediate2_tsne_data, labels, axs[0, 1], n_curves = N_CURVES)
ut.plot_representations(intermediate2_tsne_data_test, labels_test, axs[1, 1], n_curves = N_CURVES)

output_tsne_data, output2_tsne_data_test = ut.get_tsne(outputs, data_test=outputs_test, n_curves = N_CURVES)
ut.plot_representations(output_tsne_data, labels, axs[0, 2], n_curves = N_CURVES)
ut.plot_representations(output2_tsne_data_test, labels_test, axs[1, 2], n_curves = N_CURVES)

In [ ]:
#fig, ax = plt.subplots(1, 1, figsize=(15,15))
#curves, labels, probs_train = ut.get_predictions(net, train_loader_pred, device)
#pred_labels = probs_train.argmax(1, keepdim = True)
#ut.plot_confusion_matrix(np.round(labels), pred_labels, ax)


In [ ]:
pred_labels

In [ ]:
curves, labels, probs_train_sample = ut.get_predictions(net, train_loader, device)

In [ ]:
#N_WEIGHTS = 25
#weights = net.fc2.weight.data
#plot_weights(weights, N_WEIGHTS)

In [ ]:
weights1 = net.fc1.weight.data
w1 = weights1.cpu().numpy().reshape(-1,1)
weights2 = net.fc2.weight.data
w2 = weights2.cpu().numpy().reshape(-1,1)
weights3 = net.fc3.weight.data
w3 = weights3.cpu().numpy().reshape(-1,1)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,15))
ax1.hist(w1, color='black')
ax1.set_xlabel('Layer 1')
ax2.hist(w2, color='black')
ax2.set_xlabel('Layer 2')
ax3.hist(w3, color='black')
ax3.set_xlabel('Layer 3')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,15))
curves, labels, probs_train = ut.get_predictions(net, train_loader_pred, device)
pred_labels = torch.argmax(probs_train, 1)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax1)
curves, labels, probs_test = ut.get_predictions(net, test_loader_pred, device)
pred_labels = torch.argmax(probs_test, 1)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax2)